In [1]:
import os
from openai import OpenAI
client = OpenAI(api_key="sk-proj-*****")

In [2]:
assistatnt = client.beta.assistants.create(
    model="gpt-3.5-turbo",
    instructions="You are a customer support chatbot for Advanced Recovery System a behavioral health company. Use your knowledge base to best respond to user inquiries.",
    name="ARS Chatbot",
    tools=[{"type": "file_search"}],
)

In [3]:
vector_store = client.beta.vector_stores.create(name="ARS Locations and Programs")

with open("ars.md", "rb") as file:
    file_batch = client.beta.vector_stores.files.upload_and_poll(
        vector_store_id=vector_store.id, file=file
    )

print(file_batch.status)

completed


In [4]:
assistatnt = client.beta.assistants.update(
    assistant_id=assistatnt.id,
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    },
)

In [5]:
thread = client.beta.threads.create()

thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What programs do you offer?",
)

In [6]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistatnt.id,
    instructions="Only user the customer question. If no information is found, please tell the user that you are unable to find the information and offer to connect them with a representative.",
)

In [7]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
message_content = messages[0].content[0].text
print(message_content.value)

The programs offered by Advanced Recovery Systems include medical detox, inpatient/residential rehab, partial hospitalization, outpatient rehab, intensive outpatient programs, individual and group therapy, teletherapy, medication-assisted treatment, and various amenities at different locations such as fitness facilities, basketball courts, swimming pools, and therapeutic activities like yoga and art. The facilities offering these programs include The Recovery Village Ridgefield, The Recovery Village Umatilla, Orlando Recovery Center, The Recovery Village Palm Beach at Baptist Health, The Recovery Village Palmer Lake, Denver Mental Health & Counseling, and more. Each facility provides specific levels of care and amenities tailored to the needs of patients seeking treatment for substance use disorders and co-occurring mental health conditions.【4:0†source】【4:1†source】【4:2†source】.
